In [1]:
import chainer
from chainer.functions import caffe
from chainer import cuda
import chainer.functions as F
from chainer.functions import caffe
from chainer import cuda, Function, FunctionSet, gradient_check, Variable, optimizers

In [2]:
import json
import unicodecsv as csv
import pickle
import numpy as np
import cv2

In [3]:
from gensim import corpora

In [4]:
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
with open('../data/relationships-half.json', 'r') as f:
    all_relationships = json.load(f)
with open('../data/attributes-half.json', 'r') as f:
    all_attributes = json.load(f)

In [6]:
with open('../work/attribute_train_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    attribute_train_image_ids=reader.next()
    attribute_train_image_ids = map(int,attribute_train_image_ids)
    
with open('../work/attribute_val_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    attribute_val_image_ids=reader.next()
    attribute_val_image_ids = map(int,attribute_val_image_ids)
    
with open('../work/relationship_train_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    relationship_train_image_ids=reader.next()
    relationship_train_image_ids = map(int,relationship_train_image_ids)
    
with open('../work/relationship_val_image_ids.csv', 'r') as f:
    reader = csv.reader(f)
    relationship_val_image_ids=reader.next()
    relationship_val_image_ids = map(int,relationship_val_image_ids)

In [7]:
def get_image_relationships(image_id,all_relationships):
    for i in xrange(len(all_relationships)):
        image_id_=all_relationships[i][u'id']
        if image_id==image_id_:
            relationships=all_relationships[i][u'relationships']
    return relationships

def get_image_attributes(image_id,all_attributes):
    for i in xrange(len(all_attributes)):
        image_id_=all_objects[i][u'id']
        if image_id==image_id_:
            attributes=all_attributes[i][u'attributes']
    return attributes

In [ ]:
#Caffeモデルをロード
print "loading caffe models"
#Load Caffe Model
cnn_model_place="../chainer_caption_generation/data/bvlc_googlenet_caffe_chainer.pkl"
with open(cnn_model_place, 'r') as f:
    func = pickle.load(f)

def feature_exractor(self,x_chainer_variable): #to extract image feature by CNN.
    y, = self.func(inputs={'data': x_chainer_variable}, outputs=['pool5/7x7_s1'],
                  disable=['loss1/ave_pool', 'loss2/ave_pool','loss3/classifier'],
                  train=False)
    return y

In [ ]:
train_predicates=[]
for image_id in relationship_train_image_ids:
    image_relationships=get_image_relationships(image_id,all_relationships)
    for triple in image_relationships:
        triple_id=triple[u'id']
        triple_predicate=triple[u'predicate']
        train_predicates.append(triple_predicate)

In [ ]:
image_id=relationship_train_image_ids[0]
image_relationships=get_image_relationships(image_id,all_relationships)
for triple in image_relationships:
    triple_id=triple[u'id']
    triple_predicate=triple[u'predicate']

    triple_subject=triple[u'subject']
    triple_subject_name=triple_subject['name']
    triple_subject_x=triple_subject['x']
    triple_subject_y=triple_subject['y']
    triple_subject_w=triple_subject['w']
    triple_subject_h=triple_subject['h']

    triple_object=triple[u'object']
    triple_object_name=triple_object['name']
    triple_object_x=triple_object['x']
    triple_object_y=triple_object['y']
    triple_object_w=triple_object['w']
    triple_object_h=triple_object['h']

In [ ]:
triple

In [ ]:
img_file="../data/images/%d.jpg"%image_id
img=cv2.imread(img_file)

In [ ]:
img.shape

In [ ]:
img_sbj=img[triple_subject_y:triple_subject_y+triple_subject_h, triple_subject_x:triple_subject_x+triple_subject_w]
img_obj=img[triple_object_y:triple_object_y+triple_object_h, triple_object_x:triple_object_x+triple_object_w]

In [ ]:
plt.axis("off")
plt.imshow(cv2.cvtColor(img_sbj, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
dictionary = corpora.Dictionary(texts)
dictionary.filter_extremes(no_below=5, no_above=1.0)
dictionary.compactify() # remove gaps in id sequence after words that were removed
index2token = dict((v, k) for k, v in dictionary.token2id.iteritems())
ukn_id=len(dictionary.token2id)
index2token[ukn_id]='<UKN>'
